<img src="../imgs/head.png" style="width:1000px;">


# <center> Detección de Hate Speech en Twitter en Colombia</center>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
import re, nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


%matplotlib inline

## <center> Dataset </center>

In [3]:
import pandas as pd
import tweepy
import jsonpickle
def tweets_to_df(archivo):
    
    tweets = list(open(archivo, 'rt'))
    
    text = []
    weekday = []
    month = []
    day = []
    hour = []
    hashtag = []
    url = []
    favorite = []
    reply = []
    retweet = []
    follower = []
    following = []
    user = []
    screen_name = []
    idrtw=[]

    for t in tweets:
        t = jsonpickle.decode(t)
        
        # Text
        text.append(t['full_text'])
        
        idrtw.append(t['id'])
        
        # Decompose date
        date = t['created_at']
        weekday.append(date.split(' ')[0])
        month.append(date.split(' ')[1])
        day.append(date.split(' ')[2])
        
        time = date.split(' ')[3].split(':')
        hour.append(time[0]) 
        
        # Has hashtag
        if len(t['entities']['hashtags']) == 0:
            hashtag.append(0)
        else:
            hashtag.append(1)
            
        # Has url
        if len(t['entities']['urls']) == 0:
            url.append(0)
        else:
            url.append(1)
            
        # Number of favs
        favorite.append(t['favorite_count'])
        
        # Is reply?
        if t['in_reply_to_status_id'] == None:
            reply.append(0)
        else:
            reply.append(1)       
        
        # Retweets count
        retweet.append(t['retweet_count'])
        
        # Followers number
        follower.append(t['user']['followers_count'])
        
        # Following number
        following.append(t['user']['friends_count'])
        
        # Add user
        user.append(t['user']['name'])

        # Add screen name
        screen_name.append(t['user']['screen_name'])
        
    d = {'text': text,
         'weekday': weekday,
         'month' : month,
         'day': day,
         'hour' : hour,
         'has_hashtag': hashtag,
         'has_url': url,
         'fav_count': favorite,
         'is_reply': reply,
         'retweet_count': retweet,
         'followers': follower,
         'following' : following,
         'user': user,
         'screen_name' : screen_name,
         'id' : idrtw
        }
    
    return pd.DataFrame(data = d)

In [4]:
tweets1 = tweets_to_df('colombia1.json')
tweets2 = tweets_to_df('colombia2.json')
tweets3 = tweets_to_df('colombia3.json')
tweets4 = tweets_to_df('colombia4.json')
tweets5 = tweets_to_df('colombia5.json')
tweets6 = tweets_to_df('colombia6.json')
tweets7 = tweets_to_df('colombia7.json')
tweets8 = tweets_to_df('colombia8.json')
tweets9 = tweets_to_df('colombia9.json')
tweets10 = tweets_to_df('colombia10.json')
tweets11 = tweets_to_df('colombia11.json')
tweets12 = tweets_to_df('colombia12.json')
tweets13 = tweets_to_df('colombia13.json')
tweets14 = tweets_to_df('colombia14.json')
tweets15 = tweets_to_df('colombia15.json')
tweets16 = tweets_to_df('colombia16.json')
tweets17 = tweets_to_df('colombia17.json')
tweets18 = tweets_to_df('colombia18.json')
tweets19 = tweets_to_df('colombia19.json')

In [32]:
PP = pd.concat([tweets1, tweets2, tweets3, tweets4, tweets5, tweets6, tweets7, tweets8, tweets9, tweets10, tweets11, tweets12, tweets13, tweets14, tweets15, tweets16, tweets17, tweets18, tweets19])

In [33]:
tweets_df=pd.DataFrame({'text':PP['text'].unique()})
df=pd.DataFrame(tweets_df.iloc[:,0])
pd.set_option('display.max_colwidth', -1)

In [57]:
print(df.shape)
df=df.reindex(np.random.permutation(df.index))
df.head()

(16044, 1)


,text
2672,RT @meridius_maximo: @marianoig007 Después de El Putero #ElPuntero viene El Tigre Huevón jeje\n#ElTigreVeron
7673,"@MelukLeCuenta Si no logra ver la diferencia entre Samuel, Petro y Peñalosa usted es un mamerto, Petroño graduado y con maestría."
10641,@Claudiashein Ay señora valla caminando de noche por Azcapotzalco sin guardias sin lame culos y después hablamos
6380,RT @DonIzquierdo_: Voté por Santos por defender la paz y sin haber empuñado un arma nunca\n\nVoté por Petro por un mejor país y sin ser castr…
12648,@Shaankraa cerda


## <center> Preprocesamiento de los datos </center>

<img src="../imgs/preprocessing.png" style="width:800px;">

### Eliminación de ruido

In [58]:
import unicodedata
def remove_accents(string, accents=('COMBINING ACUTE ACCENT', 'COMBINING GRAVE ACCENT')):
    accents = set(map(unicodedata.lookup, accents))
    chars = [c for c in unicodedata.normalize('NFD', string) if c not in accents]
    return unicodedata.normalize('NFC', ''.join(chars))

In [59]:
#noacentos=pd.Series([remove_accents(i) for i in df['text']])
dfna=pd.DataFrame({'text':[remove_accents(i) for i in df['text']]})

In [60]:
##preprocesamiento del dataset original
clean1 = dfna["text"].str.replace((r'@[\w]*'), '')
clean2 = clean1.str.replace(r'RT', '')
clean = clean2.str.replace(r'[^a-zA-Z +^'']', '')


### Tokenización

La tokenización es el paso en el que se divide cadenas de texto más largas en piezas más pequeñas o tokens. 

In [61]:
tokens = [] 
for i in clean:
    tokens.append(nltk.word_tokenize(i))
    

### Eliminación de stopwords

In [62]:
stop_words= set(stopwords.words('spanish'))
wordnet_lemmatizer = WordNetLemmatizer()

lower_case = [[x.casefold() for x in sublst] for sublst in tokens]
cleanTweets = [[wordsub for wordsub in word if wordsub not in stop_words] for word in lower_case]


### Clasificación no supervisada

In [63]:
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### TfidfVectorizer

In [65]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words=stop_words)
vectorized_data = tfidf_vectorizer.fit_transform(clean)


In [66]:
from sklearn.cluster import KMeans 
kmeans = KMeans(n_clusters=3, n_init=10)  

In [67]:
kmeans.fit(vectorized_data) 

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [68]:
est=kmeans.predict(vectorized_data)

array([2, 2, 2, ..., 2, 2, 2], dtype=int32)

In [70]:
nosup = pd.DataFrame({'cluster':est,'tweet':clean})

In [71]:
## CANTIDAD DE TWEETS POR CLUSTER
print('0:', sum(1*(nosup['cluster']==0)), '\n 1:', sum(1*(nosup['cluster']==1)),'\n 2:', sum(1*(nosup['cluster']==2)))

0: 1505 
 1: 660 
 2: 13879


### CountVectorizer

In [64]:
count_vectorizer = CountVectorizer(ngram_range=(1,2))
vectorized_data1 = count_vectorizer.fit_transform(clean)

In [72]:
kmeans.fit(vectorized_data1) 

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [74]:
est2=kmeans.predict(vectorized_data1)
est2

array([1, 1, 1, ..., 1, 1, 0], dtype=int32)

In [76]:
nosup2 = pd.DataFrame({'cluster':est2,'tweet':clean})
print('0:', sum(1*(nosup2['cluster']==0)), '\n 1:', sum(1*(nosup2['cluster']==1)),'\n 2:', sum(1*(nosup2['cluster']==2)))

0: 2463 
 1: 11014 
 2: 2567


In [85]:
nosup2[nosup2['cluster']==0].head(5)

,cluster,tweet
13,0,Lo que omite mencionar el hoy blanqueado corroncho italiano es que Pastrana no ha sido muy claro en las explicaciones de las circunstancias de su viaje en ese avion Obvio que no se le puede sealar de la nada pero tampoco que guarde tanto silencio httpstcoPQAMIyDfH
25,0,Cual es el miedo Si su papa es tan honorable pues que lo demuestre uds no pueden esconder los pecados del gobierno Santos detras del proceso de pazSi ud Martin es tan critico pues tambien opine sobre odebrecht o sobre los falsos positivoso se siente impedido para hacerlo httpstcoOWYFLxRNM
34,0,Metase en lo Suyo que es el senado que de por si lo hace muy Mal de FotografoHay si como dijo DiomedesNo sea sapo lambon Mar
36,0,Esto es lo que va ir hacer para que llore mamerto de pacotilla httpstcocgccEoToR
41,0,Callese bobo hp que ut solo hace las cosas pa caerle bien a la gente y eso me dan ganas de vomitar todo el intestino que asco usted malparido estupido


In [86]:
nosup2[nosup2['cluster']==1].head(5)

,cluster,tweet
0,1,Despues de El Putero ElPuntero viene El Tigre Huevon jejeElTigreVeron
1,1,Si no logra ver la diferencia entre Samuel Petro y Pealosa usted es un mamerto Petroo graduado y con maestria
2,1,Ay seora valla caminando de noche por Azcapotzalco sin guardias sin lame culos y despues hablamos
3,1,Vote por Santos por defender la paz y sin haber empuado un arma nuncaVote por Petro por un mejor pais y sin ser castr
4,1,cerda


In [87]:
nosup2[nosup2['cluster']==2].head(5)

,cluster,tweet
5,2,Guerra civil de que bobo si hay algo cobarde es un uribestia tiran la piedra esconden la mano amenaza
8,2,Ajaaaa paisa como la bandeja que venden en la esquina de su casa en Bosa perdedor malparido httpstcouXTBZuExD
11,2,Cry es el nuevo disco de CAS Dios que emocion tan brava poderlo ver maana y el miercoles Seguro mas de una lagrima se asomara por mi cara La vida habla y uno le responde con musica Fokiu men gonorrea
12,2,Quizas porque es tan inutil como vos para este pais doa Paloma Un violador condenado a aos no suele estar vivo para rehacer su dao entre tanto su politiqueria de circo busca solo ingenuos votos de su ejercito de idiotas utiles
16,2,Hoy en mensajes amables de amigos sinceros Es que no debiste dejar de ser una malparida


In [88]:
indexed_dataT = hstack((np.array(range(0,vectorized_data1.shape[0]))[:,None], vectorized_data1))
indexed_dataT.shape

(16044, 143023)